In [28]:
import pymysql.cursors
import pandas as pd
import plotly.plotly as py
import plotly
from plotly.graph_objs import *
plotly.tools.set_credentials_file(username='walsky', api_key='MAJewRCxpcPsovPrsBvU')

py.sign_in("walsky", "MAJewRCxpcPsovPrsBvU")

In [29]:
py.sign_in?

In [30]:
# 连接mysql

connection = pymysql.connect(host = 'localhost',
                             user = 'walsky',
                             password = 'wal139491007',
                             db = 'world',
                             charset='utf8mb4',
                             cursorclass = pymysql.cursors.DictCursor
                             )
try:
    with connection.cursor() as cursor:
        sql = 'select Name, Continent, Population, LifeExpectancy, GNP from Country'
        cursor.execute(sql)
        result = cursor.fetchall()
finally:
    connection.close()

In [31]:
print(type(result))

<class 'list'>


In [32]:
df1 = pd.DataFrame(result)

In [33]:
df1.columns

Index(['Continent', 'GNP', 'LifeExpectancy', 'Name', 'Population'], dtype='object')

In [34]:
df = pd.DataFrame( result )
df = df[['Name','Continent','Population','LifeExpectancy','GNP']]
df = df.sort_values(['LifeExpectancy'],ascending=[1])

In [35]:
df.head()

Name Continent  Population  LifeExpectancy     GNP
237      Zambia    Africa     9169000            37.2  3377.0
143  Mozambique    Africa    19680000            37.5  2891.0
148      Malawi    Africa    10925000            37.6  1687.0
238    Zimbabwe    Africa    11669000            37.8  5951.0
2        Angola    Africa    12878000            38.3  6648.0

In [36]:
df.columns

Index(['Name', 'Continent', 'Population', 'LifeExpectancy', 'GNP'], dtype='object')

In [37]:
country_names = df['Name']

In [39]:
trace1 = Scatter(
    x=df['GNP'],
    y=df['LifeExpectancy'],
    text=country_names,
    mode='markers'
)
layout = Layout(
    title='Life expectancy vs GNP from MySQL world database',
    xaxis=XAxis( type='log', title='GNP' ),
    yaxis=YAxis( title='Life expectancy' ),
)
data = Data([trace1])
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='world GNP vs life expectancy')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~walsky/0 or inside your plot.ly account where it is named 'world GNP vs life expectancy'


In [41]:
size = df['Population']
print(type(size))

<class 'pandas.core.series.Series'>


In [49]:
sizemode = 'area'
# 设置size为area代替diameter（直径），这样显得气泡图更大
sizeref=df['Population'].max()/1e2**2
# 最大气泡区域将在100像素的量级


colors = {
    'Asia':"rgb(255,65,54)",
    'Europe':"rgb(133,20,75)",
    'Africa':"rgb(255, 244, 53)",
    'North America':"rgb(255,133,27)",
    'South America':"rgb(23,190,207)",
    'Antarctica':"rgb(61,152,112)",
    'Oceania':"rgb(255,220,0)",
}

# 定义一个鼠标悬停产生字体函数
def make_text(X):
    return 'Country: %s\
    <br>Life Expectancy: %s years\
    <br>Population: %s million'\
    % (X['Name'], X['LifeExpectancy'], X['Population']/1e6)

# 定义一个足记函数 返回Scatter对象re

def make_trace(X, continent, sizes, color):
    return Scatter(x =X['GNP'], #GNO on the x-xaxis
                  y = X['LifeExpectancy'],
                  name=continent, # hover时的大陆名
                  mode = 'makers',
                  text=X.apply(make_text, axis=1).tolist(),
                  marker =Marker(color = color,
                              size = sizes, # make size ,size is list
                              sizeref = sizeref,
                              sizemode = sizemode,
                              opacity = 0.6,
                              line = Line(width=3,color="white")# borders
                              ))
# 初始化Data对象
data = Data()

# 数据按照大陆子dataframe分组 (命名为X)
# 每个大陆制作一个跟踪对象并附加到Data对象

for continent, X, in df.groupby('Continent'):
    sizes = X['Population']   #get population array
    color = colors[continent] #get color
    
    data.append(make_trace(X, continent, sizes, color))
    
# Set plot and axis titles
title = "Life expectancy vs GNP from MySQL world database (bubble chart)"
x_title = "Gross National Product"
y_title = "Life Expectancy [in years]"

# Define a dictionary of axis style options
axis_style = dict(  
    type='log',
    zeroline=False,       # remove thick zero line
    gridcolor='#FFFFFF',  # white grid lines
    ticks='outside',      # draw ticks outside axes 
    ticklen=8,            # tick length
    tickwidth=1.5         #   and width
)

# Make layout object
layout = Layout(
    title=title,             # set plot title
    plot_bgcolor='#EFECEA',  # set plot color to grey
    hovermode="closest",
    xaxis=XAxis(
        axis_style,      # add axis style dictionary
        title=x_title,   # x-axis title
        range=[2.0,7.2], # log of min and max x limits
    ),
    yaxis=YAxis(
        axis_style,      # add axis style dictionary
        title=y_title,   # y-axis title
    )
)

# Make Figure object
fig = Figure(data=data, layout=layout)

# (@) Send to Plotly and show in notebook
py.iplot(fig, filename='s3_life-gdp')
    

In [50]:
# (!) Set 'size' values to be proportional to rendered area,
#     instead of diameter. This makes the range of bubble sizes smaller
sizemode='area'       

# (!) Set a reference for 'size' values (i.e. a population-to-pixel scaling).
#     Here the max bubble area will be on the order of 100 pixels
sizeref=df['Population'].max()/1e2**2

colors = {
    'Asia':"rgb(255,65,54)", 
    'Europe':"rgb(133,20,75)",
    'Africa':"rgb(0,116,217)",
    'North America':"rgb(255,133,27)",
    'South America':"rgb(23,190,207)",
    'Antarctica':"rgb(61,153,112)",
    'Oceania':"rgb(255,220,0)",
}

# Define a hover-text generating function (returns a list of strings)
def make_text(X):
    return 'Country: %s\
    <br>Life Expectancy: %s years\
    <br>Population: %s million'\
    % (X['Name'], X['LifeExpectancy'], X['Population']/1e6)  

# Define a trace-generating function (returns a Scatter object)
def make_trace(X, continent, sizes, color):  
    return Scatter(
        x=X['GNP'],  # GDP on the x-xaxis
        y=X['LifeExpectancy'],    # life Exp on th y-axis
        name=continent,    # label continent names on hover
        mode='markers',    # (!) point markers only on this plot
        text=X.apply(make_text, axis=1).tolist(),
        marker= Marker(
            color=color,           # marker color
            size=sizes,            # (!) marker sizes (sizes is a list)
            sizeref=sizeref,       # link sizeref
            sizemode=sizemode,     # link sizemode
            opacity=0.6,           # (!) partly transparent markers
            line= Line(width=3,color="white")  # marker borders
        )
    )

# Initialize data object 
data = Data()

# Group data frame by continent sub-dataframe (named X), 
#   make one trace object per continent and append to data object
for continent, X in df.groupby('Continent'):
    
    sizes = X['Population']                 # get population array 
    color = colors[continent]               # get bubble color
    
    data.append(
        make_trace(X, continent, sizes, color)  # append trace to data object
    ) 

    # Set plot and axis titles
title = "Life expectancy vs GNP from MySQL world database (bubble chart)"
x_title = "Gross National Product"
y_title = "Life Expectancy [in years]"

# Define a dictionary of axis style options
axis_style = dict(  
    type='log',
    zeroline=False,       # remove thick zero line
    gridcolor='#FFFFFF',  # white grid lines
    ticks='outside',      # draw ticks outside axes 
    ticklen=8,            # tick length
    tickwidth=1.5         #   and width
)

# Make layout object
layout = Layout(
    title=title,             # set plot title
    plot_bgcolor='#EFECEA',  # set plot color to grey
    hovermode="closest",
    xaxis=XAxis(
        axis_style,      # add axis style dictionary
        title=x_title,   # x-axis title
        range=[2.0,7.2], # log of min and max x limits
    ),
    yaxis=YAxis(
        axis_style,      # add axis style dictionary
        title=y_title,   # y-axis title
    )
)

# Make Figure object
fig = Figure(data=data, layout=layout)

# (@) Send to Plotly and show in notebook
py.iplot(fig, filename='s3_life-gdp')